- 단일 테이블 쿼리에서 실행자는 계획 트리 끝부터 루트까지 순서대로 계획 노드를 가져온 다음 해당 노드의 처리를 수행하는 함수를 호출.

- 각 계획 노드에는 해당 작업을 실행하기 위한 기능이 있음.
  - 이러한 함수는 src/backend/executor/ 디렉터리에 있음 .

- 예를들어
  - 순차 스캔(ScanScan)을 실행하는 함수는 nodeSeqscan.c 에 정의되어 있음.
  - 인덱스 스캔(IndexScanNode)을 실행하는 함수는 nodeIndexscan.c 에 정의되어 있음.
  - SortNode를 정렬하는 함수는 nodeSort.c 에 정의되어 있.

- 실행 프로그램의 수행 방식을 이해하는 가장 좋은 방법은 EXPLAIN 명령의 출력을 읽는 것.
  - PostgreSQL의 EXPLAIN은 계획 트리를 거의 그대로 보여줍니다.


```
testdb=# EXPLAIN SELECT * FROM tbl_1 WHERE id < 300 ORDER BY data;
                          QUERY PLAN                           
---------------------------------------------------------------
 Sort  (cost=182.34..183.09 rows=300 width=8)
   Sort Key: data
   ->  Seq Scan on tbl_1  (cost=0.00..170.00 rows=300 width=8)
         Filter: (id < 300)
(4 rows)
```



- Executor가 어떻게 수행되는지 살펴보겠습니다. EXPLAIN 명령의 결과를 맨 아래 줄부터 맨 위 줄까지 읽음.
  - 6행 : 처음에 실행기는 nodeSeqscan.c 에 정의된 함수를 사용하여 순차 스캔 작업을 수행 .
  - 4행 : 다음으로 실행기는 nodeSort.c 에 정의된 함수를 사용하여 순차 스캔 결과를 정렬 .

**임시 파일**
- Executor는 질의 처리를 위해 메모리에 할당된 work_men과 temp_buffers를 사용하지만, 메모리 내에서만 처리가 불가능한 경우에는 임시 파일을 사용.

- ANALYZE 옵션을 사용하면 EXPLAIN 명령이 실제로 쿼리를 실행하고 실제 행 수, 실제 런타임 및 실제 메모리 사용량을 표시


```
testdb=# EXPLAIN ANALYZE SELECT id, data FROM tbl_25m ORDER BY id;
                                                        QUERY PLAN                                                        
--------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=3944070.01..3945895.01 rows=730000 width=4104) (actual time=885.648..1033.746 rows=730000 loops=1)
   Sort Key: id
   Sort Method: external sort  Disk: 10000kB
   ->  Seq Scan on tbl_25m  (cost=0.00..10531.00 rows=730000 width=4104) (actual time=0.024..102.548 rows=730000 loops=1)
 Planning time: 1.548 ms
 Execution time: 1109.571 ms
(6 rows)
```

- 6행에서 EXPLAIN 명령은 실행기가 크기가 10000kB인 임시 파일을 사용했음을 보여줌.

- 임시 파일은 base/pg_tmp 하위 디렉터리에 일시적으로 생성되며, 명명 방법은 다음과 같음

```
{"pgsql_tmp"} + {PID of the postgres process which creates the file} . {sequencial number from 0}
```

- 예를 들어 임시 파일 'pgsql_tmp8903.5'는 pid가 8903인 postgres 프로세스에서 생성된 6번째 임시 파일.

```
$ ls -la /usr/local/pgsql/data/base/pgsql_tmp*
-rw-------  1 postgres  postgres  10240000 12  4 14:18 pgsql_tmp8903.5
```